# TensorFlow・Kerasのトレーニング♨

## CNN - 初歩的なCNN 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [CNNアルゴリズム・モデル](#RNNアルゴリズム・モデル)
  - [MINST-DNN](#MINST-DNN)
  - [MINST-CNN](#MINST-CNN)
  - [商品の自動タグ付け](#商品の自動タグ付け)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## [環境準備](TensorFlowAndKeras0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

import keras
print(keras.__version__)
# モデル定義
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### 共通関数

#### 画像確認

##### 画像とラベルの確認

In [ ]:
def show_image_info(x, y, label, index):
    print("label: ", label[y[index]])
    print("Image: ")
    plt.imshow(x[index].astype(np.uint8))
    plt.show()

##### ランダムな画像一覧

In [ ]:
def show_image_list(x, y, numOfCls=10, numOfImgInCls=10):
    pos = 1
    plt.figure(figsize=(numOfCls, numOfImgInCls))

    for targetClass in range(numOfCls):
        targetIdx = []
        
        # クラスclassIDの画像のインデックスリストを取得
        for i in range(len(y)):
            if y[i] == targetClass:
                targetIdx.append(i)
        
        # 各クラスからランダムに選んだ最初の10個の画像を描画
        np.random.shuffle(targetIdx)
        for idx in targetIdx[:numOfImgInCls]:
            plt.subplot(numOfCls, numOfImgInCls, pos)
            plt.imshow(x[idx])
            plt.axis('off')
            pos += 1

plt.show()

##### 誤った推論の画像を表示する関数

In [ ]:
def show_incorrect_image_list(x, y, y_pred, label, numOfImg=10):
    index = (y != y_pred)
    for i, val in enumerate(index):
        if val == True:
            print('predict: ', label[y_pred[i]])
            print('answer : ', label[y[i]])
            show_image_info(x, y, label, i)
            numOfImg -= 1
            if numOfImg <= 0:
                break

#### [分類問題関連](ScikitLearnTraining5.ipynb)

##### 分類問題のメトリック表示関数

In [ ]:
def print_metrics(label, pred):
    print('accuracy: %.3f' % metrics.accuracy_score(label, pred)) # 正答率
    
    print('\nmicro') # ミクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='micro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='micro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='micro')) # f値
    
    print('\nmacro') # マクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='macro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='macro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='macro')) # f値

##### 混同行列のグラフ化関数

In [ ]:
def plot_cm(confmat, label):
    numOfCls = len(label)
    fig, ax = plt.subplots(figsize=(numOfCls, numOfCls))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
            
    # 軸目盛を打つ場所を決める
    ax.set_xticks(np.arange(len(label)))
    ax.set_yticks(np.arange(len(label)))
    # 軸目盛を設定
    ax.set_xticklabels(label)
    ax.set_yticklabels(label)
    #plt.xticks(np.array(label)) # x軸の目盛りを指定
    #plt.yticks(np.array(label)) # y軸の目盛りを指定
    
    plt.xlabel('y_pred label')
    plt.ylabel('y label')
    plt.show()

#### 学習履歴表示関数

##### 損失

In [ ]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

##### 正解率

In [ ]:
def plot_history_acc(hist):
    plt.plot(hist.history['accuracy'],label="accuracy for training")
    plt.plot(hist.history['val_accuracy'],label="accuracy for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.show()

## CNNアルゴリズム・モデル

### MINST-DNN

#### データ

##### 生成

In [ ]:
from keras.datasets import mnist
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data()
print(x_train_org.shape, x_test_org.shape)
print(y_train_org.shape, y_test_org.shape)

##### 加工
一旦、結合します。

In [ ]:
x_org = np.vstack([x_train_org, x_test_org])
y_org = np.concatenate([y_train_org, y_test_org])
print(x_org.shape, y_org.shape)

##### 理解

In [ ]:
show_image_info(x_org, y_org, [0,1,2,3,4,5,6,7,8,9], 10)

In [ ]:
show_image_list(x_org, y_org, 10, 10)

##### 準備

###### XのKeras入力用型変換

In [ ]:
x_std = x_org.astype('f')

###### Xの画素を0.0-1.0の範囲に正規化

In [ ]:
x_std /= 255

###### 正解ラベルのOne-Hotエンコーディング
Keras（≒ 深層学習）では正解ラベルはOne-Hotベクトル化が必要。

In [ ]:
# エンコーディング
yy = np_utils.to_categorical(y_org, num_classes=10).astype('i') 
# デコーディング
print((yy.argmax(axis=1) == y_org).all())

###### 学習・テストデータの分割

In [ ]:
x_train = x_std[:60000]
x_test = x_std[60000:]
y_train = yy[:60000]
y_test = yy[60000:]
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

#### モデリング

##### DNNの定義
これは、CNNではなくDNN
- 入力層  
入力ベクトルの要素数だけ。
- 隠れ層が
  - 3つ
  - 900, 1000, 500ノード
  - 活性化関数はrelu
- 出力層  
多クラス分類なので
  - ノードは分類の数だけ
  - 活性化関数はsoftmax

In [ ]:
# モデルのインスタンスを作成
model = Sequential()

# addメソッドで層を追加していく。

# 入力層
# Flatten：入力の変換層、入力サイズを指定
model.add(Flatten(input_shape=(28, 28)))

# 多層化：繰り返し
model.add(Dense(900))
model.add(Activation('relu'))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))

# 出力層：
# Dense：全結合（線形結合）層、出力サイズを指定
model.add(Dense(10)) # 0-9の分類なので
# Activation: 活性化関数を定義（softmax
model.add(Activation('softmax'))

##### コンパイル
- 多値分類の損失関数は交差エントロピー（categorical_crossentropy）
- [optimizer=SGD](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは正解率（accuracy）

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])

##### 確認

In [ ]:
model.summary()

#### 実行

##### 学習
DNNのMINSTを並のPCで実行すると5分位かかる。

```Python
batch_size = 100
n_epoch = 20
# Keras Model の fit() は History オブジェクトを返す
hist = model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)
```

###### モデルの保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#model.save('../work/minst-dnn.h5')

###### モデルの復元

In [ ]:
model = load_model('../work/minst-dnn.h5')

##### 推論

In [ ]:
index = 10
show_image_info(x_org, y_org, [0,1,2,3,4,5,6,7,8,9], index)

predict = model.predict(x_std[index].reshape(1, 28, 28)).argmax()
answer  = y_org[index]

print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解')
else:
    print('不正解')

In [ ]:
yy_pred = model.predict(x_std)

#### 評価

##### 実測・予測を表示

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=1), dtype=np.int64)
ret = (y_org == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
print(model.metrics_names)

In [ ]:
score = model.evaluate(x_std, yy)
print('loss:', score[0])
print('accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

##### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_minst = confusion_matrix(y_org, y_pred)
plot_cm(cm_minst, [0,1,2,3,4,5,6,7,8,9])

###### 混同行列のメトリクス

In [ ]:
print_metrics(y_org, y_pred)

###### 間違った組み合わせの表示

In [ ]:
show_incorrect_image_list(x_org, y_org, y_pred, [0,1,2,3,4,5,6,7,8,9], 5)

##### 学習履歴を表示

In [ ]:
#plot_history_loss(hist)

In [ ]:
#plot_history_acc(hist)

### MINST-CNN

### データ
[MINST-DNN](#MINST-DNN)と同じ。

#### モデリング

##### CNNの定義

In [ ]:
model = Sequential()

# 畳み込み１
model.add(Conv2D(32, # フィルター数
                 input_shape=(28, 28, 1),
                 kernel_size=(3, 3),
                 activation='relu'))
# 畳み込み２
model.add(Conv2D(64, # フィルター数
                 kernel_size=(3, 3),
                 activation='relu'))
# プーリング１
model.add(MaxPool2D(pool_size=(2, 2)))

# 画像からベクトル（畳み込み＆プーリング → 全結合の所に挿入
model.add(Flatten())

# Dropoutの追加位置
# 無効化比率0.25のDropout
model.add(Dropout(0.25))

# 全結合層１
model.add(Dense(128, activation='relu'))

# Dropoutの追加位置
# 無効化比率0.5のDropout
model.add(Dropout(0.5))

# 最終層は無効化しない

# 全結合層２
model.add(Dense(10, activation='softmax'))

##### コンパイル
- 多値分類の損失関数は交差エントロピー（categorical_crossentropy）
- [optimizer=SGD](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは正解率（accuracy）

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

##### 確認

In [ ]:
model.summary()

#### 実行

##### 学習
CNNのMINSTを並のPCで実行すると20分位かかる（のでload_modelする。

```python
batch_size = 100
n_epoch = 20
# Keras Model の fit() は History オブジェクトを返す
hist = model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)
```

###### モデルの保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#model.save('../work/minst-cnn.h5')

###### モデルの復元

In [ ]:
model = load_model('../work/minst-cnn.h5')

##### 推論

In [ ]:
index = 10
show_image_info(x_org, y_org, [0,1,2,3,4,5,6,7,8,9], index)

predict = model.predict(x_std[index].reshape(1, 28, 28)).argmax()
answer  = y_org[index]

print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解')
else:
    print('不正解')

In [ ]:
yy_pred = model.predict(x_std)

#### 評価

##### 実測・予測を表示

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=1), dtype=np.int64)
ret = (y_org == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
score = model.evaluate(x_std, yy)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

##### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_minst = confusion_matrix(y_org, y_pred)
plot_cm(cm_minst, [0,1,2,3,4,5,6,7,8,9])

###### 混同行列のメトリクス

In [ ]:
print_metrics(y_org, y_pred)

###### 間違った組み合わせの表示

In [ ]:
show_incorrect_image_list(x_org, y_org, y_pred, [0,1,2,3,4,5,6,7,8,9], 5)

##### 学習履歴を表示

In [ ]:
#plot_history_loss(hist)

In [ ]:
#plot_history_acc(hist)

### 商品の自動タグ付け

#### データ

##### 生成

In [ ]:
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
from keras.datasets import cifar10
(x_train_org, y_train_org), (x_test_org, y_test_org) = cifar10.load_data()
print(x_train_org.shape, x_test_org.shape)
print(y_train_org.shape, y_test_org.shape)

##### 加工

###### 一旦、結合します。

In [ ]:
x_org = np.vstack([x_train_org, x_test_org])
y_org = np.concatenate([y_train_org, y_test_org])
print(x_org.shape, y_org.shape)

##### 理解

In [ ]:
show_image_info(x_org, y_org.flatten(), [0,1,2,3,4,5,6,7,8,9], 10)

In [ ]:
show_image_list(x_org, y_org.flatten(), 10, 10)

##### 準備

###### 要素の並び順を入れ替える
（Opencv, Kerasの仕様の関係上

In [ ]:
# サンプル数, height, width, channelへ変更
#x_org = x_org.transpose([0, 2, 3, 1])

###### XのKeras入力用型変換

In [ ]:
x_std = x_org.astype('f')

###### Xの画素を0.0-1.0の範囲に正規化

In [ ]:
x_std /= 255

###### 正解ラベルのOne-Hotエンコーディング
Kerasでは正解ラベルはOne-Hotベクトル化が必要。

In [ ]:
# エンコーディング
yy = np_utils.to_categorical(y_org.flatten(), num_classes=10).astype('i') 
# デコーディング
print((yy.argmax(axis=1) == y_org.flatten()).all())

###### 学習・テストデータの分割

In [ ]:
x_train = x_std[:50000]
x_test = x_std[50000:]
y_train = yy[:50000]
y_test = yy[50000:]
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

#### モデリング

##### CNNの定義
- 入力層  
入力ベクトルの要素数だけ。
- 隠れ層が
  - 畳み込み層とプーリング層1-3
  - 全結合層1-2
- 出力層  
多クラス分類なので
  - ノードは分類の数だけ
  - 活性化関数はsoftmax

In [ ]:
model = Sequential()

# 畳み込み層とプーリング層１ 
## 畳み込み層
model.add(Conv2D(
    input_shape=(32, 32, 3), # 入力層のサイズを決める
    filters=64,              # フィルタ数
    kernel_size=(4, 4),      # カーネル・サイズ
    strides=(1, 1),          # ストライド
    padding='same'))         # パディング
## プーリング増
model.add(MaxPool2D(pool_size=(2, 2))) 
model.add(Activation('relu'))

# 畳み込み層とプーリング層２
## 畳み込み層
model.add(Conv2D(
    filters=128,
    kernel_size=(4, 4),
    strides=(1, 1),
    padding='same'))
## プーリング増
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Activation('relu'))

# 畳み込み層とプーリング層３
## 畳み込み層
model.add(Conv2D(
    filters=128,
    kernel_size=(4, 4),
    strides=(1, 1),
    padding='same'))
## プーリング増
model.add(MaxPool2D(pool_size=(2, 2))) 
model.add(Activation('relu'))

# 画像からベクトル（畳み込み＆プーリング → 全結合の所に挿入
model.add(Flatten())

# 全結合層１
model.add(Dense(512))
model.add(Activation('relu'))

# Dropoutの追加位置
## 無効化比率0.5のDropout
#model.add(Dropout(rate=0.5))
## 最終層は無効化しない

# 全結合層２
model.add(Dense(10))
model.add(Activation('softmax'))

##### コンパイル
- 多値分類の損失関数は交差エントロピー（categorical_crossentropy）
- [optimizer=SGD](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは正解率（accuracy）

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(0.01), # 学習率：0.01
              metrics=['accuracy'])

##### 確認

In [ ]:
model.summary()

#### 実行

##### 学習
並のPCで実行すると数時間かかる（GPUが必要

```python
batch_size = 500
n_epoch = 30
hist = model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)
```

###### モデルの保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#model.save('../work/cifar10-cnn.h5')

###### モデルの復元

In [ ]:
model = load_model('../work/cifar10-cnn.h5')

##### 推論

In [ ]:
index = 99
show_image_info(x_org, y_org.flatten(), ['飛','車','鳥','猫','鹿','犬','蛙','馬','船','貨'], index)

predict = model.predict(x_std[index].reshape(1, 32, 32, 3)).argmax()
answer  = y_org[index]

print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解')
else:
    print('不正解')

In [ ]:
yy_pred = model.predict(x_std)

#### 評価

##### 実測・予測を表示

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=-1), dtype=np.uint8)
ret = (y_org.flatten() == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
score = model.evaluate(x_std, yy)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

##### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_cifar10 = confusion_matrix(y_org, y_pred)
plot_cm(cm_cifar10, 
       ['plane','car','bird','cat','deer','dog','frog','horse','ship','wagon'])

###### 混同行列のメトリクス

In [ ]:
print_metrics(y_org, y_pred)

###### 間違った組み合わせの表示

In [ ]:
show_incorrect_image_list(
    x_org, y_org.flatten(), y_pred,
    ['飛','車','鳥','猫','鹿','犬','蛙','馬','船','貨'], 5)

##### 学習履歴を表示

In [ ]:
#plot_history_loss(hist)

In [ ]:
#plot_history_acc(hist)